# 2.1 点估计-极大似然法
## 2.1.1 极大似然法的概念
## 2.1.2 连续函数空间的解析解

In [1]:
import numpy as np
import scipy.stats as st
from  collections import namedtuple 

#均值为2，标准差为10的正态分布随机变量1000个
x = st.norm.rvs(loc=2,scale=10,size=1000)
print('前10个值：',x[0:10])

#通过上述对数似然方程的估计公式，计算均值和标准差
mu = np.sum(x)/len(x)
sigma = np.sqrt(np.sum((x-np.mean(x))**2)/len(x))
print('\n使用极大似然法估计得到的均值mu：%0.2f，标准差sigma为：%0.2f。'%(mu,sigma))

前10个值： [ 2.69137236 11.75849144 -4.97270788 -7.07326982  1.42206366 -0.09344527
  8.39577773  1.52678488 15.44926397  1.2740186 ]

使用极大似然法估计得到的均值mu：1.66，标准差sigma为：10.06。


## 2.1.3 对数似然方程的数值解

以柯西(Couchy)分布为例,该总体分布的概率密度函数的尺度参数为γ，位置参数θ。

当尺度参数为1，位置参数为0时，称之为柯西分布的标准化形式或标准柯西分布。

In [2]:
'''
使用scipy的fsolve和root函数求似然方程的数值解
fsolve和root函数都需要使用样本数据的中位数作为求解的起始点
基于的思路都是不断地迭代
'''

from scipy.optimize import fsolve,root
#生成模拟数据
data = st.cauchy.rvs(loc=15,scale=1,size = 100)
#对数似然方程
def func(theta,x):
    return np.sum((x-theta)/(1+(x-theta)**2))
result = fsolve(func,np.median(data),args=data)
print('fsolve函数求解：',result)
result1 = root(func,np.median(data),args=data)
print('root函数求解',result1.x)
print('模拟数据的中位数：',np.median(data))

fsolve函数求解： [14.83009993]
root函数求解 [14.83009993]
模拟数据的中位数： 14.84033238399855


In [3]:
# 牛顿法
def cauchy_mle(x,theta):
    sum = 0
    for i in range(0,len(x)):
        sum = sum+(x[i]-theta)/(1+(x[i]-theta)**2)
    return sum

#对数似然函数的一阶导数
def cauchy_mle_der(x,theta):
    sum = 0  
    for i in range(0,len(x)):
        tmp = (x[i]-theta)**2
        sum = sum+(tmp-1)/(1+tmp)**2
    return sum

#用样本中位数作为初始估计值
theta=np.median(data)

#迭代次数控制，当前一次迭代值与当前迭代值小于0.001时停止迭代。
#说明已经收敛。
std_diff=1e-18

while True:##无限循环
    ###牛顿法的迭代公式
    theta1 = theta - cauchy_mle(data,theta)/cauchy_mle_der(data,theta)
    ###当本次迭代的数值解和上次迭代数值解之差的绝对值小于阈值时，停止迭代，
    if np.abs(theta1-theta)<=std_diff:
        break
    else:
        theta=theta1
    print(theta)

14.830092310643128
14.830099934030185
14.830099934034045
